In [1]:
# Это нужно, чтобы можно было работать с файлами в Django
import sys 
import os, django

# Домашний комп
if os.path.isdir("C:\\IT\\python\\django_sbx\\django_sbx\\django_sbx"):
    sys.path.append("C:\\IT\\python\\django_sbx\\django_sbx\\django_sbx")
# Рабочий комп 
elif os.path.isdir("C:\\Users\\rustem.diveev\\Desktop\\django_sbx\\django_sbx"):
    sys.path.append("C:\\Users\\rustem.diveev\\Desktop\\django_sbx\\django_sbx")
    
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "django_sbx.settings")
django.setup()

# Это нужно, чтобы разрешить синхронные операции в асинхронном окружении - ничего не понял
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
from django.db import IntegrityError 

from models_tutorial.models import (
    Person, BlankNullModel, DefaultModel, PkModel
)

# Очистка моделей
Person.objects.all().delete()
BlankNullModel.objects.all().delete()
DefaultModel.objects.all().delete()

(0, {})

In [3]:
p = Person(name="Fred Flintstone", shirt_size="L")
p.save()

In [4]:
p.shirt_size

'L'

In [5]:
# Обращение к display value у поля, где есть choices 
p.get_shirt_size_display()

'Large'

In [6]:
# Так-то можно сохранить любое значение, получается
p = Person(name="Jumbo the Hutt", shirt_size="Z")
p.save()

In [7]:
p.shirt_size

'Z'

In [8]:
# Но будет показываться то же самое, если нет сопоставления
p.get_shirt_size_display()

'Z'

In [9]:
"""
    Проверка работы параметров blank и null у поля модели 
    blank - разрешает хранить пустые значения (на форме, на БД не влияет как выяснилось позже)
    null - хранит пустые значения как null 
"""
# Если null=False, a blank=True, то None передавать нельзя
try:
    BlankNullModel(
        name="None",
        null=None,
        blank=None,
        blank_null=None
    ).save()
except IntegrityError as err:
    print(err)

ОШИБКА:  значение NULL в столбце "blank" отношения "models_tutorial_blanknullmodel" нарушает ограничение NOT NULL
DETAIL:  Ошибочная строка содержит (1, None, , null, null, null).



In [10]:
"""
    В БД Postgres null, blank_null - null
    Для поля null=False - если не указывать поле в конструкторе - то запишется как пустая строка
"""

p = BlankNullModel(
    name="None_except_blank_true_field",
    null=None,
    blank_null=None
)
p.save()

In [11]:
p.null is None

True

In [12]:
p.blank_null is None

True

In [13]:
"""
    В БД - везде пустые строки
"""
p = BlankNullModel(
    name="Empty_string",
    not_null_not_blank="",
    null="",
    blank="",
    blank_null=""
)
p.save()

In [14]:
p.null

''

In [15]:
p.blank

''

In [16]:
p.blank_null

''

In [17]:
"""
    Тоже интересно, если не указывать поля в конструкторе, где null=True, 
    то в бд запишется null, а не пустая строка, где null=False
"""
p = BlankNullModel(
    name="diff between blank false and true",
    not_null_not_blank="",
    blank=""
)
p.save()

In [18]:
p.not_null_not_blank

''

In [19]:
# Получается для postgres нет разницы между blank=True и blank=False
# https://stackoverflow.com/questions/8609192/what-is-the-difference-between-null-true-and-blank-true-in-django/8609425#8609425
p.blank

''

In [20]:
p.null is None

True

In [21]:
p.blank_null is None

True

In [22]:
"""
    Проверка default - можно передать как значение, так и callable 
"""
d = DefaultModel()
d.save()

In [23]:
d.default

'TEST'

In [24]:
d.default_callable

'LITERAL'

In [25]:
d.default_various_callable

99

In [26]:
"""
    Кул, работает - можно задать функцию в python, возвращающую разные значения по умолчанию 
    Посмотрел через Dbeawer - не увидел через Quick DDL - что создаются конструкции default
    Но это не точно
"""
for i in range(10):
    DefaultModel().save()
    
for i in DefaultModel.objects.all():
    print(i.default_various_callable)

99
22
72
31
62
18
97
76
4
69
74


In [32]:
"""
    Проверка PK
    Значение PK менять нельзя - если получить существующий объект, 
    а затем изменить его PK, то при сохранении создасться новый объект
    
"""
vars(Person)

mappingproxy({'__module__': 'models_tutorial.models',
              'SHIRT_SIZES': [('S', 'Small'), ('M', 'Medium'), ('L', 'Large')],
              '__doc__': 'Person(id, name, shirt_size)',
              '_meta': <Options for Person>,
              'DoesNotExist': models_tutorial.models.Person.DoesNotExist,
              'MultipleObjectsReturned': models_tutorial.models.Person.MultipleObjectsReturned,
              'name': <django.db.models.query_utils.DeferredAttribute at 0x1eadbe04580>,
              'shirt_size': <django.db.models.query_utils.DeferredAttribute at 0x1eadbe045e0>,
              'get_shirt_size_display': functools.partialmethod(<function Model._get_FIELD_display at 0x000001EAD9E31900>, , field=<django.db.models.fields.CharField: shirt_size>),
              'id': <django.db.models.query_utils.DeferredAttribute at 0x1eadbe04760>,
              'objects': <django.db.models.manager.ManagerDescriptor at 0x1eadbe04700>})

In [33]:
"""
    Если primary_key - не указывается, то создается models.BigAutoField
"""
vars(Person.id)

{'field': <django.db.models.fields.BigAutoField: id>}

In [34]:
vars(Person.id.field)

{'name': 'id',
 'verbose_name': 'ID',
 '_verbose_name': 'ID',
 'primary_key': True,
 'max_length': None,
 '_unique': False,
 'blank': True,
 'null': False,
 'remote_field': None,
 'is_relation': False,
 'default': django.db.models.fields.NOT_PROVIDED,
 'editable': True,
 'serialize': False,
 'unique_for_date': None,
 'unique_for_month': None,
 'unique_for_year': None,
 'choices': None,
 'help_text': '',
 'db_index': False,
 'db_column': None,
 '_db_tablespace': None,
 'auto_created': True,
 'creation_counter': -16,
 '_validators': [],
 '_error_messages': None,
 'error_messages': {'invalid_choice': 'Value %(value)r is not a valid choice.',
  'null': 'This field cannot be null.',
  'blank': 'This field cannot be blank.',
  'unique': '%(model_name)s with this %(field_label)s already exists.',
  'unique_for_date': '%(field_label)s must be unique for %(date_field_label)s %(lookup_type)s.',
  'invalid': '“%(value)s” value must be an integer.'},
 'attname': 'id',
 'column': 'id',
 'concrete':